In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [3]:
df = pd.read_csv("../data/HIGGS.csv",delimiter=',',header=None)


In [4]:
X = df.iloc[:,1:].to_numpy().astype("float")
y = df.iloc[:,0].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
seeds = seeds[::-1]
train_size = 0.01
test_size = 0.001
dataset = 'higgs'
# X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.01,test_size=0.001,stratify=y)

In [5]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

def initial(y_one_hot):
    init_log_odds = np.sum(y_one_hot,axis=0)/y_one_hot.shape[0]
    
    init_p = np.exp(init_log_odds)
    init_p = init_p/(1+init_p)

    return init_log_odds,init_p

# model

In [6]:
import sys
del sys.modules['GTGP'] 
del sys.modules['Engine'] 
del sys.modules['Functions'] 

import Functions
from Engine import Engine
from GTGP import GTGP

In [11]:
learning_rate=0.1
max_depth = 3
bins=8
lam=100

gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

total_size = 10
elite_size = 10
epoch = 1000
gp_epoch = 3
verbose = 1
tolerance=0.0001

gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

retrain_epoch=100
alpha=1
beta=1
gammer=1

verbose=1
gtgp.lam = 1
gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

19 71 1
35 137 2
52 194 3
66 250 4
80 290 5
91 331 6
101 357 7
110 384 8
121 421 9
129 449 10
135 469 11
144 494 12
159 561 13
165 579 14
173 603 15


KeyboardInterrupt: 

In [7]:
def fit_trees():
    learning_rate=0.1
    max_depth = 4
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size = 10
    elite_size = 10
    epoch = 10
    gp_epoch = 3
    verbose = 1
    tolerance=0.0001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=10
    alpha=1
    beta=1
    gammer=1

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,test_size=test_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_roc = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    test_roc = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)


    # with open('./benchmark/'+dataset+'.csv','a') as f:
    #     s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
    #     f.writelines(s)

17 73 1
34 142 2
47 203 3
59 239 4
69 269 5
80 320 6
84 332 7
98 392 8
112 442 9
120 468 10
	residual_bin:  0.006997346878051758
	cover_bin:  0.005999565124511719
	grad_bin:  0.0
log grads 0.0
p1 0.0
p1_2 0.0009999275207519531
log grads 0.0
p1 0.0
p1_2 0.0
	residual_bin:  0.006000041961669922
	cover_bin:  0.005999088287353516
	grad_bin:  0.0
log grads 0.0010004043579101562
p1 0.00099945068359375
p1_2 0.0010004043579101562
log grads 0.0
p1 0.0
p1_2 0.0
	residual_bin:  0.00601649284362793
	cover_bin:  0.005983114242553711
	grad_bin:  0.0
log grads 0.0
p1 0.001980304718017578
p1_2 0.0
log grads 0.0
p1 0.0
p1_2 0.0
	residual_bin:  0.0060002803802490234
	cover_bin:  0.0060155391693115234
	grad_bin:  0.0
log grads 0.0
p1 0.0010182857513427734
p1_2 0.0009829998016357422
log grads 0.0
p1 0.0
p1_2 0.0
	residual_bin:  0.005999088287353516
	cover_bin:  0.006999969482421875
	grad_bin:  0.0
log grads 0.001001119613647461
p1 0.0009992122650146484
p1_2 0.0010006427764892578
log grads 0.0
p1 0.0
p1_2 

In [ ]:
    learning_rate=0.3
    max_depth = 4
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size = 100
    elite_size = 30
    epoch = 100
    gp_epoch = 3
    verbose = 1
    tolerance=0.0001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

85 503 1
168 1016 2
240 1434 3
313 1857 4
377 2239 5
442 2606 6
505 2933 7
564 3278 8
611 3553 9
668 3914 10
721 4257 11
767 4495 12
812 4774 13
845 4951 14
888 5242 15
944 5634 16
984 5868 17
1021 6113 18
1055 6297 19
1095 6535 20
1128 6746 21
1161 6919 22
1199 7161 23
1235 7377 24
1265 7555 25
1309 7875 26
1340 8088 27
1370 8256 28
1397 8421 29
1425 8547 30
1447 8679 31
1474 8860 32
1500 9020 33
1540 9300 34
1573 9517 35
1607 9761 36
1638 9980 37
1677 10223 38
1711 10437 39
1746 10620 40
1770 10784 41
1796 10942 42
1824 11106 43
1846 11206 44
1878 11434 45
1897 11535 46
1928 11728 47
1962 11940 48
1980 12052 49
2008 12236 50
2027 12333 51
2057 12497 52
2084 12682 53
2124 12982 54
2150 13146 55
2175 13313 56
2193 13413 57
2211 13541 58
2239 13721 59
2254 13820 60
2285 14035 61
2305 14121 62
2322 14214 63
2346 14358 64
2377 14559 65
2404 14736 66
2427 14877 67
2448 15018 68
2461 15077 69
2490 15276 70
2509 15387 71
2537 15587 72
2553 15683 73
2573 15793 74
2599 15977 75
2617 16077 76
2

In [ ]:
p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

(0.8565519523822079, 0.8060525930155569)

In [ ]:
retrain_epoch=100
alpha=1
beta=1
gammer=1

verbose=1
gtgp.lam = 10000
train_acc,test_acc,train_sse,test_sse = gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)


# verbose=0
# gtgp.retrain_estimators(retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

retrain  1 :
	train: 0.7283272727272727 39361.669897527325 	test: 0.7167272727272728 4073.5264339413225
retrain  2 :
	train: 0.7385090909090909 38177.666662836855 	test: 0.7214545454545455 4004.672995664143
retrain  3 :
	train: 0.7442545454545455 37459.57915794868 	test: 0.723 3975.2365078031694
retrain  4 :
	train: 0.7486 36919.97439983066 	test: 0.7250909090909091 3959.3274886338977
retrain  5 :
	train: 0.7524090909090909 36476.714226178934 	test: 0.7262727272727273 3950.003833332743
retrain  6 :
	train: 0.7557818181818182 36095.124944888565 	test: 0.7281818181818182 3944.401845799943
retrain  7 :
	train: 0.7590818181818182 35757.72432194795 	test: 0.7268181818181818 3941.243315991411
retrain  8 :
	train: 0.7615454545454545 35453.89931170898 	test: 0.727909090909091 3939.8012437453253
retrain  9 :
	train: 0.7635454545454545 35177.040533771535 	test: 0.7281818181818182 3939.5892069284364
retrain  10 :
	train: 0.7652909090909091 34922.14823387019 	test: 0.7285454545454545 3940.32051273

KeyboardInterrupt: 

In [13]:
gtgp.print_model()

p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

----------------GTGP-------------
Number of Trees: 696
Average of depth: 1.3635057471264367
Number of nodes: 2682


(0.8303415336436182, 0.787721114749806)

In [ ]:
gtgp.print_model()

----------------GTGP-------------
Number of Trees: 120
Number of nodes: 410


In [ ]:
p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

(0.8001366067252442, 0.7896649628178634)

In [ ]:
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

              precision    recall  f1-score   support

           0    0.70700   0.68721   0.69696     51709
           1    0.72925   0.74735   0.73819     58291

    accuracy                        0.71908    110000
   macro avg    0.71812   0.71728   0.71758    110000
weighted avg    0.71879   0.71908   0.71881    110000

              precision    recall  f1-score   support

           0    0.69702   0.68246   0.68966      5171
           1    0.72343   0.73683   0.73007      5829

    accuracy                        0.71127     11000
   macro avg    0.71022   0.70965   0.70987     11000
weighted avg    0.71101   0.71127   0.71107     11000



In [ ]:
gtgp.print_model()

----------------GTGP-------------
Number of Trees: 1703
Number of nodes: 8271


In [ ]:
p = gtgp.train_p
test_p = gtgp.test_p
y_test,y_test_one_hot = gtgp.to_one_hot(y_test)
roc_auc_score(gtgp.y_one_hot.toarray(),(p.T/np.sum(p,axis=1)).T),roc_auc_score(y_test_one_hot.toarray(),(test_p.T/np.sum(test_p,axis=1)).T)

(0.8370318754478703, 0.8032409787365098)

In [ ]:
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

              precision    recall  f1-score   support

           0    0.74663   0.72426   0.73528     51709
           1    0.76173   0.78197   0.77172     58291

    accuracy                        0.75485    110000
   macro avg    0.75418   0.75312   0.75350    110000
weighted avg    0.75463   0.75485   0.75459    110000

              precision    recall  f1-score   support

           0    0.71727   0.69619   0.70658      5171
           1    0.73733   0.75656   0.74682      5829

    accuracy                        0.72818     11000
   macro avg    0.72730   0.72638   0.72670     11000
weighted avg    0.72790   0.72818   0.72790     11000



# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,test_size=test_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [ ]:
import xgboost


In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,test_size=test_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=3,n_estimators=100)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
    xgb = xgboost.XGBClassifier()
    xgb.fit(X_train,y_train)

    import json

    def item_generator(json_input, lookup_key):
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key:
                    yield v
                else:
                    yield from item_generator(v, lookup_key)
        elif isinstance(json_input, list):
            for item in json_input:
                yield from item_generator(item, lookup_key)

    def tree_depth(json_text):
        json_input = json.loads(json_text)
        depths = list(item_generator(json_input, 'depth'))
        return max(depths) + 1 if len(depths) != 0 else 1

    train_acc = accuracy_score(y_train,xgb.predict(X_train))
    test_acc = accuracy_score(y_test,xgb.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

    booster = xgb.get_booster()

    tree_df = booster.trees_to_dataframe()
    depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
    num_trees = len(depths)
    depth = np.average(depths)
    num_nodes = len(tree_df)

    with open('./benchmark_xgb/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,test_size=test_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    clf = GradientBoostingClassifier(n_estimators=800)
    # clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GBDT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

KeyboardInterrupt: 

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,test_size=test_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_roc = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_roc)+","+str(test_roc)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)